<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Particionando-bases-de-treino-e-teste-com-split-70-30%" data-toc-modified-id="Particionando-bases-de-treino-e-teste-com-split-70-30%-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Particionando bases de treino e teste com split 70-30%</a></span></li><li><span><a href="#Particionando-bases-de-treino-e-teste-com-diferentes-músicas" data-toc-modified-id="Particionando-bases-de-treino-e-teste-com-diferentes-músicas-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Particionando bases de treino e teste com diferentes músicas</a></span></li></ul></div>

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight
from imblearn.under_sampling import RandomUnderSampler

In [2]:
tf.__version__

'2.2.0'

In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [4]:
df = pd.read_csv('E:\chord-detection-challenge\DataBase\CSV/status_A1.csv')
df.sort_values(by='title', inplace=True)
images_path = sorted(glob.glob('E:\chord-detection-challenge\DataBase\clean_windows/Train/*'))
df['Unnamed: 0'] = np.array(images_path)
df.reset_index(inplace=True, drop=True)
df['status'] = df['status'].astype(str)

### Particionando bases de treino e teste com split 70-30%

In [5]:
### particiona considerando 70-30% e mantendo a frequência de amostras para treino, validação e teste de acordo com as colunas título e status (rótulo da rede)

X_train, X_test, y_train, y_test = train_test_split(df[['Unnamed: 0', 'title']], df['status'], test_size=0.30, random_state=42, stratify=df[['status', 'title']])
df_train = pd.concat([X_train, y_train], axis=1)
X_train, X_val, y_train, y_val = train_test_split(df_train[['Unnamed: 0', 'title']], df_train['status'], test_size=0.30, random_state=42, stratify=df_train[['status', 'title']])

### contatena atributos de entrada e rótulo em um único dataframe para utilizar o flow_from_dataframe do tensorflow
df_test = pd.concat([X_test, y_test], axis=1)
df_train = pd.concat([X_train, y_train], axis=1)
df_val = pd.concat([X_val, y_val], axis=1)

print('Total de imagens de treinamento', len(df_train))
print('Total de imagens de validação', len(df_val))
print('Total de imagens de teste', len(df_test))

Total de imagens de treinamento 7648
Total de imagens de validação 3278
Total de imagens de teste 4683


In [6]:
undersample_train = RandomUnderSampler(sampling_strategy='majority')
undersample_validation = RandomUnderSampler(sampling_strategy='majority')

X_undertrain, y_undertrain = undersample_train.fit_resample(df_train[['Unnamed: 0', 'title']], df_train['status'])
X_undervalidation, y_undervalidation = undersample_validation.fit_resample(df_val[['Unnamed: 0', 'title']], df_val['status'])

In [7]:
df_train = pd.concat([X_undertrain, y_undertrain], axis=1)
df_val = pd.concat([X_undervalidation, y_undervalidation], axis=1)

### Particionando bases de treino e teste com diferentes músicas

In [8]:
songs, n = df['title'].unique(), 5
index = np.random.choice(len(songs), 5, replace=False)  
selected_songs = songs[index] ## seleciona n músicas disponíveis para teste
df_test = df[df['title'].isin(selected_songs)] ## banco de teste contém todos os espectrogramas das n músicas selecionadas anteriormemente
df_train = df[~(df['title'].isin(selected_songs))] ## banco de treino contém os espectrogramas de todas as músicas EXCETO as selecionadas anteriormente para teste

X_train, X_val, y_train, y_val = train_test_split(df_train[['Unnamed: 0', 'title']], df_train['status'], test_size=0.30, random_state=42, stratify=df_train[['status', 'title']]) ## divide em validação considerando 30% e balanceamento de acordo com título e status

### contatena atributos de entrada e rótulo em um único dataframe para utilizar o flow_from_dataframe do tensorflow
df_train = pd.concat([X_train, y_train], axis=1)
df_val = pd.concat([X_val, y_val], axis=1)

print('Total de imagens de treinamento', len(df_train))
print('Total de imagens de validação', len(df_val))
print('Total de imagens de teste', len(df_test))


Total de imagens de treinamento 9591
Total de imagens de validação 4111
Total de imagens de teste 1907


In [8]:
datagen=ImageDataGenerator(rescale=1./255)
train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory='E:\chord-detection-challenge\DataBase\clean_windows/Train/', x_col='Unnamed: 0', y_col="status", class_mode="binary", target_size=(224,224), batch_size=32)
valid_generator=datagen.flow_from_dataframe(dataframe=df_val, directory='E:\chord-detection-challenge\DataBase\clean_windows/Train/', x_col='Unnamed: 0', y_col="status", class_mode="binary", target_size=(224,224), batch_size=32)
test_generator=datagen.flow_from_dataframe(dataframe=df_test, directory='E:\chord-detection-challenge\DataBase\clean_windows/Train/', x_col='Unnamed: 0', y_col="status", class_mode="binary", target_size=(224,224), batch_size=1,shuffle=False)

Found 4494 validated image filenames belonging to 2 classes.
Found 1926 validated image filenames belonging to 2 classes.
Found 4683 validated image filenames belonging to 2 classes.


In [9]:
#from tensorflow.keras.models import Model
restnet = tf.keras.applications.VGG16(
    include_top=False, # não vai aproveitar a camada de saída 
    weights=None, #não pega os pesso da imagenet
    input_shape=(224,224,3)
)
output = restnet.layers[-1].output
output = tf.keras.layers.Flatten()(output)
restnet = tf.keras.Model(inputs=restnet.input, outputs=output)
for layer in restnet.layers: #treina tudo do zero
    layer.trainable = True
restnet.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
mc = tf.keras.callbacks.ModelCheckpoint('resnet_model.h5', monitor='val_binary_accuracy', mode='max', save_best_only=True)

model = tf.keras.models.Sequential()
model.add(restnet)
model.add(tf.keras.layers.Dense(128, activation='relu', input_dim=(224,224,3)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
#   tf.keras.layers.Conv2D(32, (3, 3), padding='same',
#                input_shape=(32,32,3)),
#   tf.keras.layers.MaxPool2D(),
#   tf.keras.layers.Conv2D(64, (3, 3)),
#   tf.keras.layers.Conv2D(128, (3, 3)),
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(128,activation='relu'),
#   tf.keras.layers.Dense(2)
#)

model.compile(
    optimizer=tf.keras.optimizers.Adamax(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
    #weighted_metrics=[tf.keras.metrics.BinaryAccuracy()]
)
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(df_train['status']),
                                                 df_train['status'])
class_weights = dict(enumerate(class_weights))

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    #class_weight=class_weights,
                    epochs=10,
                    callbacks = [mc])

C:\Users\cflav\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=['0' '1'], y=0       0
1       0
2       0
3       0
4       0
       ..
4489    1
4490    1
4491    1
4492    1
4493    1
Name: status, Length: 4494, dtype: object as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Epoch 1/10
140/140 [==============================] - 214s 2s/step - loss: 0.7405 - binary_accuracy: 0.5027 - val_loss: 0.6931 - val_binary_accuracy: 0.5005
Epoch 2/10
140/140 [==============================] - 168s 1s/step - loss: 0.6932 - binary_accuracy: 0.4948 - val_loss: 0.6931 - val_binary_accuracy: 0.5000
Epoch 3/10
140/140 [==============================] - 169s 1s/step - loss: 0.6932 - binary_accuracy: 0.4939 - val_loss: 0.6931 - val_binary_accuracy: 0.5000
Epoch 4/10
140/140 [==============================] - 166s 1s/step - loss: 0.6931 - binary_accuracy: 0.5054 - val_loss: 0.6931 - val_binary_accuracy: 0.5010
Epoch 5/10
140/140 [==============================] - 156s 1s/step - loss: 0.6932 - binary_accuracy: 0.4931 - val_loss: 0.6931 - val_binary_accuracy: 0.4995
Epoch 6/10
140/140 [==============================] - 159s 1s/step - loss: 0.6932 - binary_accuracy: 0.4971 - val_loss: 0.6932 - val_binary_accuracy: 0.4995
Epoch 7/10
140/140 [==============================] - 151s

In [20]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

print('---------------Teste-------------')
test_generator.reset()
predictions = model.predict(test_generator,
                        steps=STEP_SIZE_TEST,
                        verbose=1)

---------------Teste-------------
4683/4683 [==============================] - 122s 26ms/step


In [38]:
predictions

array([[0.49953243],
       [0.49953243],
       [0.49953243],
       ...,
       [0.49953243],
       [0.49953243],
       [0.49953243]], dtype=float32)

In [32]:
y_pred = predictions > 0.5

In [29]:
predicted_class_indices=np.argmax(predictions,axis=1)


In [30]:
predicted_class_indices

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [27]:
print(accuracy_score(test_generator.labels, y_pred))

0.29404228058936577


In [28]:
print(classification_report(test_generator.labels, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3306
           1       0.29      1.00      0.45      1377

    accuracy                           0.29      4683
   macro avg       0.15      0.50      0.23      4683
weighted avg       0.09      0.29      0.13      4683



C:\Users\cflav\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cflav\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cflav\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
